# Quality control

In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

2021-09-01 16:05:07 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-09-01 16:05:08 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-848846b477-8tkk6:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/qc/hail-20210901-1605-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd

Load variant data.

In [4]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/variants.mt')

Split multi-allelic variants into separate rows. This makes a lot of analyses much easier.

**FIXME:** while `split_multi_hts()` is really useful, it is causing me a lot of headaches in some other areas.

In [5]:
mt = hl.split_multi_hts(mt)

In [6]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 1:========================================>               (69 + 24) / 96]

Number of variants: 3358693
Number of samples: 474


[Stage 1:======================================================>  (92 + 4) / 96]

In [7]:
def variant_counts(mt):
    # Count number of variants, SNPs and indels. Only first allele in alternate allele list is considered.
    variant_counts_struct = mt.aggregate_rows(hl.struct(
        n_variants = hl.agg.count(),
        n_snps = hl.agg.count_where(hl.is_snp(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        n_indels = hl.agg.count_where(hl.is_indel(mt.alleles[0], mt.alleles[1])) / hl.agg.count()))
    
    variant_counts_pd = pd.DataFrame(variant_counts_struct.values(), index=variant_counts_struct.keys(), columns=[''])
    return variant_counts_pd

In [8]:
variant_counts(mt)

[Stage 2:=====================================================> (188 + 4) / 192]

,
n_variants,3.358693e+06
n_snps,4.001976e-01
n_indels,5.998024e-01


## VQSR filters

We will remove variants not passing the VQSR filters. First, however, we will look a bit closer into these values.

In [9]:
mt = mt.transmute_rows(filters=hl.delimit(mt.filters, ','))

We will calculate some mean QC statistics for each VQSR tranch.

In [10]:
# Calculate variant statistics.
mt = hl.variant_qc(mt)

# Get rows table.
rows_ht = mt.rows()

# Aggregate.
result = (rows_ht.group_by(rows_ht.filters)
         .aggregate(mean_gq = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.gq_stats.mean), hl.agg.mean(rows_ht.variant_qc.gq_stats.mean)),
                   mean_dp = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.dp_stats.mean), hl.agg.mean(rows_ht.variant_qc.dp_stats.mean)),
                   mean_af = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.AF[0]), hl.agg.mean(1 - rows_ht.variant_qc.AF[0])),
                   mean_vqslod = hl.agg.filter(~hl.is_nan(rows_ht.info.VQSLOD), hl.agg.mean(rows_ht.info.VQSLOD)),
                   n_variants = hl.agg.count()))

We convert the results to a Pandas dataframe.

In [11]:
vqsr_stats_pd = result.to_pandas()

[Stage 3:======================================================>(191 + 1) / 192]2021-09-01 16:06:15 Hail: INFO: Ordering unsorted dataset with network shuffle


Below we first print the statistics for the SNPs and then for the indels. The rows are sorted by mean genotype quality. The empty filter row (`filter=''`) corresponds to all unfiltered variants.

In [12]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheSNP99.00to99.90', 'VQSRTrancheSNP99.90to100.00', ''])].sort_values('mean_gq')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
3,VQSRTrancheSNP99.00to99.90,38.370792,18.238171,0.014862,-1.060170,535602
4,VQSRTrancheSNP99.90to100.00,41.638284,24.347085,0.012312,-15.672791,363164
0,,50.905433,22.646584,0.064824,inf,1504277


In [13]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheINDEL99.00to99.90', 'VQSRTrancheINDEL99.90to100.00', ''])].sort_values('mean_gq')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
1,VQSRTrancheINDEL99.00to99.90,39.146784,16.936859,0.003169,-1.528588,660377
0,,50.905433,22.646584,0.064824,inf,1504277
2,VQSRTrancheINDEL99.90to100.00,54.085372,26.461966,0.005288,-5.745145,295273


We will only look at the **high-quality variants** from now on. So we remove variants in the 99% to 99.9% and the 99.9% to 100% VQSR tranches, and keep only the unfiltered variants.

In [14]:
mt = mt.filter_rows(mt.filters == '')

We check how many variants we have remaining after filtering.

## Genotype QC

In [15]:
p = hl.plot.histogram(mt.GQ, range=(0, 100))
p.xaxis.axis_label = 'Genotype quality'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 8:======================================================>(191 + 1) / 192]

Filter genotypes with low quality (GQ), using GQ > 20 for SNPs and GQ > 40 for indels.

After the filter some sites may have become invariant. These are removed.

NOTE: Many (or most) of the lower quality genotypes, i.e. the low-end tail of the histogram above, are indels.

In [16]:
# Calculate variant statistics.
mt = hl.variant_qc(mt)

mt = mt.filter_entries(hl.if_else(
    hl.is_snp(mt.alleles[0],mt.alleles[1]),
    mt.GQ > 20,
    mt.GQ > 40))

### Allelic balance

Note that since we've split the table, all rows are diallelic.

We compute the allelic balance as $AB = \frac{AD[1]}{DP}$. Note that DP is equivalent to $AD[0] + AD[1]$ for a diallelic site.

In [17]:
mt = mt.annotate_entries(AB = mt.AD[1] / mt.DP)

In [18]:
hets_mt = mt.filter_entries(mt.GT.is_het())
p = hl.plot.histogram(hets_mt.AB, range=(0, 1))
p.xaxis.axis_label = 'Allelic balance'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 9:======================================================>(191 + 1) / 192]

Filter all heterozygotes with allelic balance outside the range of $]0.25;0.75[$.

In [19]:
mt = mt.filter_entries(hl.if_else(
    mt.GT.is_het(),
    (mt.AB > 0.25) & (mt.AB < 0.75),
    True))

## Variant QC

### Hardy-Weinberg Equilibrium (HWE)

Remove variants that significantly deviate from HWE. We use a p-value of $10^{-9}$ for SNPs and $10^{-6}$ for indels. Since our criteria for indels is 1000 times more stringent than for SNPs, we cannot expect the indels to carry information about population structure and inbreeding.

In [20]:
# Update variant statistics.
mt = hl.variant_qc(mt)

mt = mt.filter_rows(hl.if_else(
    hl.is_snp(mt.alleles[0], mt.alleles[1]),
    mt.variant_qc.p_value_hwe > 1e-9,
    mt.variant_qc.p_value_hwe > 1e-6))

### Average variant GQ

In [21]:
p = hl.plot.histogram(mt.variant_qc.gq_stats.mean, range=(0,100), legend='Mean GQ per variant histogram')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 10:=====================================================>(191 + 1) / 192]

## Sample QC

Below we see histograms of sample mean genotype quality and genotype depth. Most samples seem to have good depth and quality, although the deviation between samples is quite large. There are some samples with low depth and quality, but we will not worry about these.

In [22]:
mt = hl.sample_qc(mt)

In [23]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(0,100), legend='Mean Sample GQ')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 11:=====================================================>(191 + 1) / 192]

In [24]:
p = hl.plot.histogram(mt.sample_qc.dp_stats.mean, range=(0,60), legend='Mean Sample DP')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 14:=====================================================>(191 + 1) / 192]

Below is a histogram of the heterozygote/homozygote ratio.

In [25]:
p = hl.plot.histogram(mt.sample_qc.r_het_hom_var, legend='Het/hom rate')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 20:=====================================================>(191 + 1) / 192]

Looks like there are a few samples with a lot higher het/hom rate than the rest of the samples. Let's check whether this is due to poor coverage.

In [26]:
het_hom_thres = 2.3
mt = mt.annotate_cols(high_hom_het=mt.sample_qc.r_het_hom_var > het_hom_thres)

In [27]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.r_het_hom_var, xlabel='DP mean', ylabel='het/hom rate',
                    hover_fields={'Sample': mt.s}, label=mt.high_hom_het)
p.plot_width = 600
p.plot_height = 600
show(p)

[Stage 23:=====================================================>(191 + 1) / 192]

The plot above indicates these samples have similar coverage as the other samples, so that doesn't explain the high het/hom rate.

We can also check the MultiQC reports of these samples (file paths below), and we see that they are all of reasonable quality.

In [28]:
high_hethom_samples = mt.filter_cols(mt.sample_qc.r_het_hom_var > het_hom_thres).s.collect()

for sample in high_hethom_samples:
    print('/data/projects/fargen_phase_1/data/single_sample_data/{sample}/multiqc/multiqc_report.html'.format(sample=sample))

[Stage 26:=====================================================>(191 + 1) / 192]

/data/projects/fargen_phase_1/data/single_sample_data/FN000538/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN000909/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN000940/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001018/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001019/multiqc/multiqc_report.html


In the genealogy summary file (`/fargen/fargen_phase_1_utils/multi_sample/joint_genotyping/metadata/genealogy/individuals_summary.csv`), it seems that all these samples have reasonably deep roots in the Faroes. This means we have no reason to suspect this difference is due to these samples being from different populations.

These four samples most likely have high het/hom rate due to poor data quality. One potential reason for this is contamination of the sample in the lab.

We will **discard high het/hom rate samples**, as they may skew further analyses down the line.

In [29]:
mt = mt.filter_cols(mt.sample_qc.r_het_hom_var < het_hom_thres)

In [30]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 30:=====================================================>(191 + 1) / 192]

Number of variants: 1492532
Number of samples: 469


## Variant counts

In [31]:
# Remove sites that have become invariant after the AB filter.
mt = mt.filter_rows(mt.variant_qc.AC[1] > 0)

In [32]:
variant_counts(mt)

[Stage 31:=====================================================>(191 + 1) / 192]

,
n_variants,953581.000000
n_snps,0.441601
n_indels,0.558399


## Write data to file

In [33]:
if False:
    mt.write(BASE_DIR + '/data/mt/high_quality_variants.mt', overwrite=True)

## Summary

In this notebook we have:

* Filtered variants failing VQSR filter
* Filtered variants with genotype quality < 20
* Removed four samples with abnormally high het/hom rate (>3)